# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
it is classification problem of supervised learning. the output of this problem is to identify those student who need early intervention. so it is a binary classification problem, and output values are discrete.
for example, we can use 1 for those student who need it, 0 for student who don't need it.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
#print student_data

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [12]:
# TODO: Calculate number of students
n_students = np.shape(student_data)[0]
#print n_students #395
# TODO: Calculate number of features
#n_features = np.shape(student_data)[1]-1 #all the columns except the last one('passed') are features
n_features=len(student_data.columns[:-1])
#print n_features #30
# TODO: Calculate passing students
n_passed = sum(student_data['passed']=='yes')
#print n_passed #265
# TODO: Calculate failing students
n_failed = sum(student_data['passed']=='no')
#print n_failed #130
# TODO: Calculate graduation rate
grad_rate = np.mean(student_data['passed']=='yes')*100
#print grad_rate #67.09%

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [10]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()
#print y_all.head() #y_all:'yes','no'
y_all = y_all.replace(['yes', 'no'], [1, 0])
#print y_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [13]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)
    #print np.shape(output)
    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
    
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))
#print X_all.head()
print y_all.head() #y_all:0,1

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
0    0
1    0
2    1
3    1
4    1
Name: passed, dtype: int64


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [14]:
# TODO: Import any additional functionality you may need here
from sklearn import cross_validation
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train,X_test,y_train,y_test=cross_validation.train_test_split(X_all,y_all,test_size=num_test,random_state=42)
#StratifiedShuffleSplit 
# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])
#print y_train.head() #y_train:0,1
#print y_test.head() #y_test:0,1

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **
1.Three model:Support Vector Machine(SVM),Bayesian Network, Boosting
2.General Application:
A.SVM can be used as classification and regression. when using it to do classification, examples of seperate categories are divided into clear gap, where gap as wide as possible;
B.Bayesian Network is a probabilistic graphical model that represent variables and that dependencies though graph.it can representation of joint distribution, and used to classification based on the possibility.
C.Boosting can be used to improve the accuracy of any learning model as long as they are 'weak learners'.
Strenghts and Weakness:
A.SVM: strength: it can performance well where clear margin of seperation in complex domain. when doing classification, it will get maximum possible margin, so that it can have good accuracy and more confidence; weakness: it is slow in training large dataset, and not so well in dataset with lots of noise; what's more, SVM is a binary classifier. we had to use one Vs all to do classification. so it means we had to do classification for serveral times before we can get result for mulit-class classification.
B.Bayesian Network: strength: it is export in calucate possibility with the help of naive bayes; with graphical model it is cheap to calucate possibility, and get result of classification based on possibility. weakness: it assume that attributes are independent. but in really, it maybe not true. if an attribute haven't seen a label, it may result of zero possibility, which may give wrong singal.
C.Boosting: strength: it can be used to apply for many models, and help to improve accuracy. and it usually will not have overfitting.
weakness: weak learner like neural network with many layers, which may already have overfitting,it will also result of overfitting.
3.Why choose these three models:
other models like Neural Network, Decision Tree, Instance Based Learning, can also be used to performance classification. but i choose above three models, because i know that they are powerful to do classification. and i want to apply them in real problem, so that i can not only understand the principle, but also have hand on experience of use them, and evalution their performances.
and compare to Neural Network, i think above three selected model are simple, so they are easy to compute, while still can peformance well.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [15]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred) #, pos_label='yes': remove this. i have already transfer lable from yes,no to 1,0.


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))
    #print "==============================================================="

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [16]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier

# TODO: Initialize the three models
clf_A = SVC(random_state=42)
clf_B = GaussianNB()
clf_C = AdaBoostClassifier(random_state=42)

# TODO: Set up the training set sizes
#X_train,X_test,y_train,y_test=cross_validation.train_test_split(X_all,y_all,test_size=num_test,random_state=42)

X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 =X_train[:200]
y_train_200 =y_train[:200]

X_train_300 =X_train[:300]
y_train_300 =y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
 #train_predict(clf, X_train, y_train, X_test, y_test)
#clf_A: SVM
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

#clf_B: Naive Bayes
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

#clf_C: Boosting
#clf_B: Naive Bayes
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

#print y_train_100.head()
#print y_test.head()

Training a SVC using a training set size of 100. . .
Trained model in 0.0190 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8777.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7746.
Training a SVC using a training set size of 200. . .
Trained model in 0.0010 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8679.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7815.
Training a SVC using a training set size of 300. . .
Trained model in 0.0160 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8761.
Made predictions in 0.0150 seconds.
F1 score for test set: 0.7838.
Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0000 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8467.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.8029.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0000 seconds
Made pred

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - ?**  
#SVM
| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0010              | 0.0010                 |     0.8777       | 0.7746          |
| 200               |     0.0030              | 0.0020                 |     0.8679       | 0.7815          |
| 300               |     0.0050              | 0.0020                 |     0.8761       | 0.7838          |

** Classifer 2 - ?**  
#GaussianNB 
| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.0000               |  0.0000                |   0.8467         |  0.8029         |
| 200               |    0.0010               |  0.0010                |   0.8406         |  0.7244         |
| 300               |    0.0010               |  0.0000                |   0.8038         |  0.7634         |

** Classifer 3 - ?**  
#AdaBoostClassifier 
| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0030                |  0.0040                |  0.9481          |  0.7669         |
| 200               |   0.0060                |  0.0040                |  0.8927          |  0.8281         |
| 300               |   0.0050                |  0.0040                |  0.8637          |  0.7820         |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
i will choose SVM as the best model among these three modes for this dataset. 
here we have dataset with 395 records available,only three modes available, and performance is measure by F1 Score.the main purpose of our model is to have an ability to make a good prediction, and generilize well on new dataset(or unseen data). and F1 Score on test set is a way to inidicate this. 
among these three models, i can see that SVM has the highest F1 score when size is 300. and although boosting has a better F1 score when size is 200, but SVM compute fast at all cases than boosting; although GaussianNB always compute fast than other two models, but it's performance as measured by F1 Score on test is fluctuant(or not so coherent, or consistent) 
(Trained SVM model with 300 training size in 0.0160 seconds, it is still in a very short time, so it's acceptable.)

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **<br/>
1.Description:Support Vector Machine is a supervised learning model, which can be used to classification and regression.<br/>
2.How it tranined: For a set of training examples with labels of each data, SVM build model assign new examples into one category or the other, making it a binary linear classifier.it will represent examples a points in sapce, mapped so that examples of separate categories are divided by a clear gap that is a wide as possible.<br/> 
3.How it make prediction: new examples are then mapped into same space, and predicted to belong to a category based on which side of the gap they fail on.<br/>
if dataset is not linear seperate, it will map data to a higher dimension, so it can turn linear problem to non-linear problem. 
4.why it calls 'Support Vector Machine'? because when doing classification and try to get the maximum margin, only those data(Vector) near the margin matters. data that is far way from margin will not affect how dataset will be seperated.<br/>
5.visualisation examples.<br/>
Support Vector Machine for classification with maximum margin with support vectors:
<img src='SVM_supportVector.png'>

When dataset is not seperatable in a dimension, it will map to a higher dimension:<br/>
<img src='data_2d_to_3d_hyperplane.png'>
<img src='data_2d_to_3d_hyperplane2.png'>

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [22]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import fbeta_score,make_scorer,accuracy_score

# TODO: Create the parameters list you wish to tune
parameters = {'kernel':['linear','poly','rbf'],'C':[1, 2, 3, 4,6,10],'gamma':[1,3, 5, 7,10,50,300,100,1000]}

# TODO: Initialize the classifier
clf =SVC(random_state=42)

# TODO: Make an f1 scoring function using 'make_scorer' 
 #f1_score(target.values, y_pred, pos_label='yes')
f1_scorer2 = make_scorer(f1_score)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,parameters,f1_scorer2)

# TODO: Fit the grid search object to the training data and find the optimal parameters

grid_obj=grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning


print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0230 seconds.
Tuned model has a training F1 score of 1.0000.
Made predictions in 0.0030 seconds.
Tuned model has a testing F1 score of 0.7742.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
F1 score for training and test is 1.0000 and 0.7742.
Compare to untuned model, F1 score on training set imporve to maximum possible value 1.0000. however, the F1 score for testing set does not improve. the F1 score using all training set for untuned model is 0.7838. 
So it means i should work out to find other parameters which may be help to improve the score. or it indicate that tunned model sometimes may be overfitting the data, or default setting of the model is already optimal.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.